In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=api_key)

In [3]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)

# documents = []
# for course_dict in docs_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)


In [4]:
# Fetch the FAQ data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

# Process the documents
documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import NotFoundError

es_client = Elasticsearch('http://localhost:9200') 
print(es_client.info())

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
try:
    response = es_client.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")
except NotFoundError:
    print(f"Index {index_name} not found, nothing to delete")

es_client.indices.create(index=index_name, body=index_settings)

{'name': 'f460ba3dbd41', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'I0b-mWe_SIeGhn3MK910qg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Deleted index: course-questions


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)


/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 47.95it/s]


In [8]:
def elastic_search(query, size=5, course=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
    if course:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": course
            }
        }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    # Step 4: Extract the _score field from the top-ranking result
    top_score = response['hits']['hits'][0]['_score']
    print("The top-ranking result _score is:", top_score)
    
    return result_docs

In [9]:
query = 'How do I execute a command in a running docker container?'

**Q3**

In [24]:
results = elastic_search(query, 5, None)
print(results)

The top-ranking result _score is: 84.050095
[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for 

**Q4**

In [11]:
course = 'machine-learning-zoomcamp'
size = 3

# Perform the search
results = elastic_search(query, size=size, course=course)
print(results)
# Check if we have at least 3 results and print the third question
if len(results) >= 3:
    third_question = results[2]['question']
    print("The third question returned by the search engine is:", third_question)
else:
    print("There are less than 3 results.")


The top-ranking result _score is: 84.050095
[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local

In [12]:
# Check if we have at least 3 results and print all questions with ranks
if results:
    for i, result in enumerate(results):
        print(f"Rank {i+1}: {result['question']}")
    if len(results) >= 3:
        third_question = results[2]['question']
        print("\nThe third question returned by the search engine is:", third_question)
    else:
        print("\nThere are less than 3 results.")
else:
    print("No results found.")

Rank 1: How do I debug a docker container?
Rank 2: How do I copy files from my local machine to docker container?
Rank 3: How do I copy files from a different folder into docker container’s working directory?

The third question returned by the search engine is: How do I copy files from a different folder into docker container’s working directory?


**Q5**

In [13]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt, context


In [14]:
def rag(query, size=3, course=None):
    search_results = elastic_search(query, size, course)
    prompt, context = build_prompt(query, search_results)
    return prompt, context

In [15]:
# Example usage to get the prompt length
query = 'How do I execute a command in a running docker container?'
course = 'machine-learning-zoomcamp'
size = 3

# Perform the search
results = elastic_search(query, size=size, course=course)

# Build the prompt using the search results
prompt, context = build_prompt(query, results)
print("Prompt is:", prompt)

# print(context)

# Calculate the length of the prompt
prompt_length = len(prompt)
print("The length of the resulting prompt is:", prompt_length)

The top-ranking result _score is: 84.050095
Prompt is: You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
d

**Q6**

In [16]:
!pip install tiktoken

In [17]:
import tiktoken

In [18]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print("The number of tokens in the resulting prompt is:", num_tokens)

The number of tokens in the resulting prompt is: 322


**Bonus**

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
answer = llm(prompt)

In [21]:
print("Output:", answer)

Output: To execute a command in a running Docker container, you need to use the `docker exec` command. Here are the steps:

1. First, find the container ID of the running container by using the `docker ps` command.
2. Once you have the container ID, you can use the following command to execute a bash shell inside the container:
   ```bash
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container. This will open up an interactive terminal session within the container where you can execute commands.


In [22]:
# Given data
input_tokens_per_request = 150
output_tokens_per_request = 250
number_of_requests = 1000
price_per_1k_input_tokens = 0.005
price_per_1k_output_tokens = 0.015

# Total tokens
total_input_tokens = input_tokens_per_request * number_of_requests
total_output_tokens = output_tokens_per_request * number_of_requests

# Cost calculation
cost_for_input_tokens = (total_input_tokens / 1000) * price_per_1k_input_tokens
cost_for_output_tokens = (total_output_tokens / 1000) * price_per_1k_output_tokens
total_cost = cost_for_input_tokens + cost_for_output_tokens

print("Total Input Tokens:", total_input_tokens)
print("Total Output Tokens:", total_output_tokens)
print("Cost for Input Tokens: $", round(cost_for_input_tokens, 2))
print("Cost for Output Tokens: $", round(cost_for_output_tokens, 2))
print("Total Cost for 1000 Requests: $", round(total_cost, 2))


Total Input Tokens: 150000
Total Output Tokens: 250000
Cost for Input Tokens: $ 0.75
Cost for Output Tokens: $ 3.75
Total Cost for 1000 Requests: $ 4.5


In [23]:
# Print the response
print("Response from OpenAI:")
print(answer)

# Calculate the number of tokens in the response
response_tokens = encoding.encode(answer)
num_response_tokens = len(response_tokens)
print("The number of tokens in the response is:", num_response_tokens)

# Given prices
price_per_1k_input_tokens = 0.005
price_per_1k_output_tokens = 0.015

# Number of requests
number_of_requests = 1000

# Calculate costs
total_input_tokens = num_tokens * number_of_requests
total_output_tokens = num_response_tokens * number_of_requests

cost_for_input_tokens = (total_input_tokens / 1000) * price_per_1k_input_tokens
cost_for_output_tokens = (total_output_tokens / 1000) * price_per_1k_output_tokens
total_cost = cost_for_input_tokens + cost_for_output_tokens

print("Total Input Tokens:", total_input_tokens)
print("Total Output Tokens:", total_output_tokens)
print("Cost for Input Tokens: $", round(cost_for_input_tokens, 2))
print("Cost for Output Tokens: $", round(cost_for_output_tokens, 2))
print("Total Cost for 1000 Requests: $", round(total_cost, 2))

Response from OpenAI:
To execute a command in a running Docker container, you need to use the `docker exec` command. Here are the steps:

1. First, find the container ID of the running container by using the `docker ps` command.
2. Once you have the container ID, you can use the following command to execute a bash shell inside the container:
   ```bash
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container. This will open up an interactive terminal session within the container where you can execute commands.
The number of tokens in the response is: 120
Total Input Tokens: 322000
Total Output Tokens: 120000
Cost for Input Tokens: $ 1.61
Cost for Output Tokens: $ 1.8
Total Cost for 1000 Requests: $ 3.41
